In [47]:
from time import time

init = time()

import re
import os
import sys
import json
import yaml
from functools import reduce
from collections import ChainMap
import subprocess
import shutil

import pandas as pd
from glob import glob
import nbformat

import jax

In [48]:
old_nb_files = glob("../../../pml-book/pml1/figure_notebooks/*")
old_nb_files[:2]

['../../../pml-book/pml1/figure_notebooks/chapter9_linear_discriminant_analysis_figures.ipynb',
 '../../../pml-book/pml1/figure_notebooks/chapter11_linear_regression_figures.ipynb']

### Detect all notebooks

In [49]:
def get_fig_wise_scripts(cells):
    prev_prev_cell, prev_cell, cell = cells
    scripts = re.findall("To reproduce this figure, click the open in colab button.*", cell["source"])
    if scripts:
        return scripts


def process_notebook(file_name):
    chap_num, chap_name = file_name.split("/")[-1].split(".")[0].split("_", 1)
    chap_num = chap_num.replace("chapter", "").zfill(2)
    chap_name = chap_name.replace("_figures", "")
    nb = nbformat.read(file_name, as_version=4)

    scripts = list(map(get_fig_wise_scripts, zip(nb["cells"], nb["cells"][1:], nb["cells"][2:])))
    scripts = list(filter(None, scripts))
    # https://stackoverflow.com/a/15714097
    return {chap_num: scripts}


notebooks = list(map(process_notebook, old_nb_files))
notebooks[:2]

[{'09': [['To reproduce this figure, click the open in colab button: <a href="https://colab.research.google.com/github/probml/probml-notebooks/blob/master/notebooks-d2l/naive_bayes_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'],
   ['To reproduce this figure, click the open in colab button: <a href="https://colab.research.google.com/github/probml/probml-notebooks/blob/master/notebooks-d2l/naive_bayes_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>']]},
 {'11': []}]

### Detect paths and map between old and new

In [51]:
find_nb = lambda f: re.findall(
    '"https://colab.research.google.com/github/probml/probml-notebooks/blob/master/(.*)" target', f
)

In [52]:
old_base = "../../../probml-notebooks/"
new_base = "../notebooks/book1/"
old_to_new = {}
for i in range(len(notebooks)):
    nb_dict = notebooks[i]
    for key, value in nb_dict.items():
        for nb in jax.tree_leaves(value):
            nb_name = find_nb(nb)[0]
            old_path = os.path.join(old_base, nb_name)
            new_path = os.path.join(new_base, str(key), nb_name.split("/")[-1])
            old_to_new.update({old_path: new_path})

In [55]:
list(old_to_new.items())[:2]

[('../../../probml-notebooks/notebooks-d2l/naive_bayes_mnist.ipynb',
  '../notebooks/book1/09/naive_bayes_mnist.ipynb'),
 ('../../../probml-notebooks/notebooks/iris_dtree.ipynb',
  '../notebooks/book1/01/iris_dtree.ipynb')]

### Move to book1 folder

In [56]:
for old_path, new_path in old_to_new.items():
    if not os.path.exists(os.path.dirname(new_path)):
        os.makedirs(os.path.dirname(new_path))
    try:
        if old_path.endswith("_torch.ipynb"):
            old_path = old_path.replace("_torch.ipynb", "_jax.ipynb")
            new_path = new_path.replace("_torch.ipynb", "_jax.ipynb")
        shutil.copy(old_path, new_path)
        print("copied", old_path, "to", new_path)
    except Exception as e:
        print(e, new_path)

copied ../../../probml-notebooks/notebooks-d2l/naive_bayes_mnist.ipynb to ../notebooks/book1/09/naive_bayes_mnist.ipynb
copied ../../../probml-notebooks/notebooks/iris_dtree.ipynb to ../notebooks/book1/01/iris_dtree.ipynb
copied ../../../probml-notebooks/notebooks-d2l/conv2d_jax.ipynb to ../notebooks/book1/14/conv2d_jax.ipynb
copied ../../../probml-notebooks/notebooks/cnn_mnist_tf.ipynb to ../notebooks/book1/14/cnn_mnist_tf.ipynb
copied ../../../probml-notebooks/notebooks/poisson_regression_insurance.ipynb to ../notebooks/book1/12/poisson_regression_insurance.ipynb
copied ../../../probml-notebooks/notebooks/mlp_mnist_tf.ipynb to ../notebooks/book1/13/mlp_mnist_tf.ipynb
copied ../../../probml-notebooks/notebooks/mlp_1d_regression_hetero_tfp.ipynb to ../notebooks/book1/13/mlp_1d_regression_hetero_tfp.ipynb
copied ../../../probml-notebooks/notebooks/activation_fun_deriv_jax.ipynb to ../notebooks/book1/13/activation_fun_deriv_jax.ipynb
copied ../../../probml-notebooks/notebooks/sparse_mlp.